<a href="https://colab.research.google.com/github/jimregan/effective-eureka/blob/master/MLM_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/awslabs/mlm-scoring

  Cloning https://github.com/awslabs/mlm-scoring to /tmp/pip-req-build-f85uyr3z
  Running command git clone -q https://github.com/awslabs/mlm-scoring /tmp/pip-req-build-f85uyr3z
     |████████████████████████████████| 245kB 4.9MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 645kB 39.0MB/s 
     |████████████████████████████████| 3.8MB 32.1MB/s 
     |████████████████████████████████| 1.1MB 42.3MB/s 
     |████████████████████████████████| 890kB 61.8MB/s 
  Created wheel for mlm: filename=mlm-0.1-cp36-none-any.whl size=37511 sha256=eed303f0fbee43a69849306011f2872954caedcb6bb1e4e05a3c88df30030bb8
  Stored in directory: /tmp/pip-ephem-wheel-cache-jnl_tc95/wheels/a9/f1/d7/ae4b7c48fe69aef69b4170ad13c5109cb499daabf027e12728
  Created wheel for gluonnlp: filename=gluonnlp-0.8.3-cp36-none-any.whl size=293540 sha256=475968e65379fdff64738849ddaf578b3457c9a60797ffcba3947c32b20f3f44
  Stored in directory: /root/.cache/pip/wheels/50/6e/32/521aa84d

In [2]:
from transformers import *
model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
nlp = pipeline('fill-mask', model=model, tokenizer=tokenizer)
vocab = None

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [4]:
!pip install mxnet-cu101mkl

     |████████████████████████████████| 712.3MB 25kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [5]:
from mlm.scorers import MLMScorer, MLMScorerPT
from mlm.models import get_pretrained
import mxnet as mx
ctxs = [mx.gpu()]
scorer = MLMScorerPT(model, vocab, tokenizer, ctxs)

/usr/local/lib/python3.6/dist-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [6]:
!wget http://poleval.pl/task1/Poleval2020Task1Eval.tar.xz

--2020-08-20 21:36:10--  http://poleval.pl/task1/Poleval2020Task1Eval.tar.xz
Resolving poleval.pl (poleval.pl)... 213.135.36.94
Connecting to poleval.pl (poleval.pl)|213.135.36.94|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12592244 (12M) [application/x-tar]
Saving to: ‘Poleval2020Task1Eval.tar.xz’

Poleval2020Task1Eva 100%[===================>]  12.01M  5.07MB/s    in 2.4s    

2020-08-20 21:36:13 (5.07 MB/s) - ‘Poleval2020Task1Eval.tar.xz’ saved [12592244/12592244]



In [7]:
!tar xf Poleval2020Task1Eval.tar.xz

In [8]:
scorer.score_sentences(["witam serdecznie"])

[-13.625729203224182]

In [9]:
def one_best(id, sentences, scores, output):
  counter = 1
  lowest = 0.0
  lowest_number = 0
  for score in scores:
    if score < lowest:
      lowest = score
      lowest_number = counter
    counter = counter + 1
  output.write("%s-%d %s\n" % (id, lowest_number + 1, sentences[lowest_number]))

In [10]:
out = open(file="/content/scored", mode="a", encoding="utf-8")
sentences = []
last_id = ''
counter = 1
with open(file="/content/Poleval2020Task1Eval/nbest.txt", mode="r", encoding="utf-8") as nbest:
  for line in nbest:
    words = line.strip().split(' ')
    id = words[0]
    base_id, cur_num = id.split('-')
    if last_id == '':
      last_id = base_id
    elif last_id != base_id:
      print("last_id (%s) and current (%s) mismatch\n" % (last_id, id))
    if int(cur_num) != counter:
      print("cur_num (%d) and counter (%d) mismatch\n" % (int(cur_num), counter))
    sentence = ' '.join(words[1:])
    sentences.append(sentence)
    if len(sentences) == 100:
      scores = scorer.score_sentences(sentences)
      one_best(base_id, sentences, scores, out)
      scores = []
      sentences = []
      counter = 1
      last_id = ''
    else:
      counter = counter + 1

RuntimeError: ignored